In [2]:
sentiments = {
    "Haystacks": -0.03,
    "Ranch sauce": 0.3,
    "Cacti Needle": -0.04,
    "Solar panels": -0.03,
    "Red Flags": -0.1,
    "VR Monocle": 0.4,
    "Quantum Coffee": -0.03,
    "Moonshine": 0.02,
    "Striped shirts": 0.5
}

In [6]:
import cvxpy as cp
import numpy as np

n = len(sentiments)

print(cp.installed_solvers())

goods = list(sentiments.keys())
values = np.array(list(sentiments.values()))

pi = cp.Variable(n, integer=True)

fee = 120 * cp.sum_squares(pi) # fee = sum(120 * pi_i^2) = 120 * sum(pi_i^2)
pnl = (values @ pi) * 10000 # move % dotted with portfolio percentage scaled by the capital

objective = cp.Maximize(pnl - fee)
constraints = [cp.norm(pi, 1) <= 100] # only constraint is that our portfolio adds up to 100

problem = cp.Problem(objective=objective, constraints=constraints)
problem.solve(solver=cp.SCIP)

total = 0

print()
print(f"best pnl: {problem.value:.3f} seashells")
print("optimal allocation:")
print("-" * 50)
print(f"{'resource':<20} {'position':<15} {'allocation':<10}")
print("-" * 50)
total = 0
for i, good in enumerate(goods):
    if abs(pi.value[i]) > 0:
        position_type = "Buy" if pi.value[i] > 0 else "Sell"
        print(f"{good:<20} {position_type:<15} {pi.value[i]:<10.3f}%")
        total += abs(pi.value[i])
print("-" * 50)
print(f"{'Total':<20} {'':<15} {total:<10.3f}%")

['CLARABEL', 'OSQP', 'SCIP', 'SCIPY', 'SCS']

best pnl: 107140.000 seashells
optimal allocation:
--------------------------------------------------
resource             position        allocation
--------------------------------------------------
Haystacks            Sell            -1.000    %
Ranch sauce          Buy             13.000    %
Cacti Needle         Sell            -2.000    %
Solar panels         Sell            -1.000    %
Red Flags            Sell            -4.000    %
VR Monocle           Buy             17.000    %
Quantum Coffee       Sell            -1.000    %
Moonshine            Buy             1.000     %
Striped shirts       Buy             21.000    %
--------------------------------------------------
Total                                61.000    %
